# Si-Ge cluster expansion workflow - part 1

This is a CASM project tutorial to generate a phase diagram using a Si-Ge binary alloy cluster expansion fit to DFT calculations. The overall workflow is split into two parts.

Topics covered in part 1:

1. **Project initialization**: Define the primitive crystal structure and allowed atoms on each crystal site
2. **Enumeration**: Enumerate crystal structures which are symmetrically distinct orderings of the atoms allowed by the prim occupation DoF
3. **Calculation**: Calculate the energies of the enumerated structures using DFT
4. **Import and mapping**: Import calculation results, mapping to orderings on the prim
5. **Set reference states**: Choose reference states to define a formation energy for each structure
6. **Query**: Query calculation properties


In [ ]:
import pathlib
import libcasm.xtal as xtal
from casm.project import Project

project_path = pathlib.Path("SiGe_occ")
project_path.mkdir(parents=True, exist_ok=True)

## Project initialization

### Specify the "prim"

A primitive crystal structure and allowed degrees of freedom (the "prim") specifies:

- lattice vectors
- crystal basis sites
- global degrees of freedom
- site degrees of freedom, including allowed occupant species on each basis site.

When combined with a choice of basis function type, order, and truncation, the prim provides all the information needed to generate cluster expansion basis functions.

Here is the prim for the Si-Ge binary alloy project, which we write to a JSON-formatted file named "prim.json":

In [ ]:
prim_data = {
    "title" : "SiGe_occ",
    "lattice_vectors" : [
        [ 0.000000000000, 2.800000000000, 2.800000000000 ], # 1st lattice vector 
        [ 2.800000000000, 0.000000000000, 2.800000000000 ], # 2nd lattice vector
        [ 2.800000000000, 2.800000000000, 0.000000000000 ], # 3rd lattice vector
    ],
    "coordinate_mode" : "Fractional",
    "basis" : [
        {
            "coordinate" : [ 0.0, 0.0, 0.0 ],
            "occupant_dof" : [ "Si", "Ge" ],
        },
        {
            "coordinate" : [ 0.25, 0.25, 0.25 ],
            "occupant_dof" : [ "Si", "Ge" ],
        }
    ],
}

with open(project_path / "prim.json", 'w') as f:
    f.write(xtal.pretty_json(prim_data))

For this particular project, the prim contains:

- **lattice_vectors**: A list of crystal lattice vectors. Units are typically Angstrom, but are ultimately determined by the method used to perform calculations. 
- **basis**: A list of crystal basis sites, including coordinate and allowed degrees of freedom. For this ZrO project, the basis sites contain:
  - **coordinate**: The location of the basis site, according to the "coordinate_mode".
  - **occupants**: A list of the possible occupant species that may reside at each site. The names are case sensitive, and “Va” is reserved for vacancies.
- **coordinate_mode**: Defines the units of basis site coordinates. May be one of:
  - "Cartesian": To specify basis coordinates using Cartesian coordinates:
    $$ r_{cart} = (x, y, z) $$
  - "Fractional" or "Direct": To specify basis coordinates defined in terms of the lattice vectors:
    $$ r_{cart} = L r_{frac}, $$
    where:
    - $r_{frac}$ are the coordinates in the fractional representation
    - $r_{cart}$ are the coordinates in the Cartesian representation
    - $L$ is the lattice as a column-vector matrix. 
  
**Note**: It is common, but not required, to use the results of a fully relaxed calculation of the structure with the default occupation values for the prim lattice vectors. The default occupation on each site is the species listed first in "occupants". For occupation cluster expansions, ideal supercells of the prim lattice are used for the initial state of DFT calculations and are the default reference for strain.

### Initialize a CASM project

A CASM project is a directory containing data related to a particular prim. The CASM project directory structure standardizes the location of various files used by multiple CASM methods. This makes it easier to perform the most common operations and easier to share a project with others.

A CASM project is initialized by defining a prim and using [Project.init TODO](TODO). This will:

1. Check if the prim has a primitive unit cell with a CASM standard lattice orientation
2. Perform a symmetry analysis
3. Generate some default directories, data, and settings
4. Perform a configuration check 


Notes:

- Project files that the user should not typically modify directly, including a copy of the prim, are stored in a hidden `.casm` sub-directory of the CASM project directory. The presense or absence of the `.casm` directory is used by CASM to detect a CASM project.


In [ ]:
project = Project.init(path=project_path)

coming: 
- Show what happens with non-primitive prim or non-standard lattice?
- Visualize the prim

## Enumeration

### Introduction

To fit a cluster expansion for the Si-Ge system, we need a set of calculated energies for Si-Ge crystal structures with various orderings to use as training data. To begin, we use CASM to enumerate symmetrically distinct [Supercell]() and [Configuration]():

- A [Supercell]() defines the three-dimensional translations that repeat a crystal structure. 

  -  A supercell can be specified by the integer transformation matrix, $T$, relating the superstructure lattice vectors, $S$, to the unit structure lattice vectors, $L$, according to $S = L T$, where $S$ and $L$ are shape=(3,3) matrices with lattice vectors as columns.
  
  TODO: figure
 
- A [Configuration]() is a compact representation of the unit cell for a crystal structure that is allowed by the DoF specified in the prim. For this Si-Ge project, a configuration can be specified by:
 
  - the supercell that is the unit cell for the crystal structure, and
  - the occupant on each sites in the supercell (i.e. is Si or Ge on each site in the supercell).
  
  TODO: figure

The Supercell object holds symmetry representations that efficiently applying symmetry operations to a configuration, allowing for comparisons and checks to determine if a configuration is symmetrically distinct. The same symmetry representations can be used to transform any configuration in the same supercell, so a Supercell object can be shared by multiple Configuration objects.

### Supercell enumeration

#### Enumerating supercells by volume

The method [enum.supercells_by_volume]() enumerates symmetrically distinct supercells from a minimum to a maximum volume, specified as integer multiples of the prim unit cell volume. It also has additional options, described in the reference documentation, for more complex use cases:

- Enumerate supercells of another supercell
- Enumerate 1d or 2d supercells
- Enumerate supercells with a fixed shape but different sizes

In [ ]:
# Enumerate supercells with volume 1 to 4
project.enum.supercells_by_volume(
    max=4, 
    min=1, 
    id="supercells_by_volume.1", 
    verbose=True,
)

#### Enumeration Data

The results of an enumeration can be accessed using the [EnumData]() class. The last enumeration is saved in [project.enum.last](). We can put additional information, including a text description of the enumeration, in the [EnumData.meta]() dict and save the updated EnumData using [commit](). Subsequently, if "desc" exists in [EnumData.meta](), it will be printed along with summary information such as the number of supercells.

In [ ]:
enum_data = project.enum.last
enum_data.meta = {"desc": "Initial supercell enumeration"}
enum_data.commit()
print(enum_data)

Later, the [EnumData]() may also be accessed by id string using the [enum.get]() method.

In [ ]:
enum_data = project.enum.get("supercells_by_volume.1")
print(enum_data)

#### SupercellSet

The supercells enumerated by [enum.supercells_by_volume]() are stored as [SupercellRecord](https://prisms-center.github.io/CASMcode_pydocs/libcasm/configuration/2.0/reference/libcasm/_autosummary/libcasm.configuration.SupercellRecord.html#supercellrecord) in a [SupercellSet](https://prisms-center.github.io/CASMcode_pydocs/libcasm/configuration/2.0/reference/libcasm/_autosummary/libcasm.configuration.SupercellSet.html#supercellset). Each SupercellRecord includes a [Supercell](https://prisms-center.github.io/CASMcode_pydocs/libcasm/configuration/2.0/reference/libcasm/_autosummary/libcasm.configuration.Supercell.html#supercell) and some additional information about the supercell, including a [supercell_name]() string which is used as an identifier.

A SupercellSet:

- does not keep multiple SupercellRecord for supercells that have the same superlattice vectors;
- does allow storing separate SupercellRecord for supercells which are distinct (have different superlattice vectors) but are symmetrically equivalent (superlattice points are mapped by a crystal group operation).

Iterating over a [SupercellSet](https://prisms-center.github.io/CASMcode_pydocs/libcasm/configuration/2.0/reference/libcasm/_autosummary/libcasm.configuration.SupercellSet.html#supercellset) yields [SupercellRecord](https://prisms-center.github.io/CASMcode_pydocs/libcasm/configuration/2.0/reference/libcasm/_autosummary/libcasm.configuration.SupercellRecord.html#supercellrecord). Each SupercellRecord includes a [Supercell](https://prisms-center.github.io/CASMcode_pydocs/libcasm/configuration/2.0/reference/libcasm/_autosummary/libcasm.configuration.Supercell.html#supercell) and some additional information about the Supercell.

In [ ]:
# Iterate over the first three SupercellRecord
# in the SupercellSet and print the record
for i, record in enumerate(project.enum.last.supercell_set):
    print(record)
    if i == 2:
        break


#### Storing multiple enumerations

Enumerations are stored in directories based on their id string. If an id is not given, or has value None, a new enumeration is automatically generated in sequential order. If the id of an existing enumeration is given, that enumeration is updated with any additional supercells generated.


In [ ]:
# Enumerate supercells with volume 3 to 5
project.enum.supercells_by_volume(max=5, min=3, id="supercells_by_volume.2")
print()
print(project.enum.last)

### Configuration enumeration

#### Enumerating configurations by supercell

The method [enum.occ_by_supercell]() enumerates all occupations in supercells ranging from a minimum to a maximum volume. All configurations are guaranteed to be in a canonical supercell. By default it:

- only outputs primitive configurations,
- only outputs configurations in canonical form (the configuration that compares greatest to all configurations in a supercell that can be mapped by symmetry operations).

With these defaults, if enumeration proceeds without skipping supercells, all symmetrically distinct configurations will be enumerated.

As with enum.supercells_by_volume it also has additional options, described in the reference documentation, for more complex use cases:

- Enumerate occupations in supercells of another supercell
- Enumerate occupations in 1d or 2d supercells
- Enumerate occupations in supercells with a fixed shape but different sizes

**Warning**: The number of possible occupations in a $n$-component alloy with $m$ sites is $n^m$. Take care not to request too large of an enumeration. 
    

In [ ]:
# Enumerate configurations in supercells with volume 1 to 4
project.enum.occ_by_supercell(
    max=4, 
    min=1, 
    id="occ_by_supercell.1",
)

#### ConfigurationSet

The configurations enumerated by [enum.occ_by_supercell]() are stored as [ConfigurationRecord](https://prisms-center.github.io/CASMcode_pydocs/libcasm/configuration/2.0/reference/libcasm/_autosummary/libcasm.configuration.ConfigurationRecord.html#configurationrecord) in a [ConfigurationSet](https://prisms-center.github.io/CASMcode_pydocs/libcasm/configuration/2.0/reference/libcasm/_autosummary/libcasm.configuration.ConfigurationSet.html#configurationset). Each ConfigurationRecord includes a [Configuration](https://prisms-center.github.io/CASMcode_pydocs/libcasm/configuration/2.0/reference/libcasm/_autosummary/libcasm.configuration.Configuration.html#configuration) and some additional information about the configuration, including a [configuration_name]() string which is used as an identifier.

A ConfigurationSet:

- requires configuration be in a canonical supercell;
- does not keep multiple ConfigurationRecord for configurations that have the same DoF values;
- does allow storing separate ConfigurationRecord for configurations which are distinct (have different DoF values) but are symmetrically equivalent (DoF values are mapped by a symmetry operation).
- users are responsible for placing any other constraints (canonical configurations only, primitive configurations only, etc.) on which configuration are added to ConfigurationSet.

**Warning**: ConfigurationSet is optimized for finding unique configurations using canonical supercells. Users must ensure that configuration added to ConfigurationSet are in a canonical supercell. This is not checked by ConfigurationSet but required to ensure proper configuration naming, serialization, and deserialization. Configurations that are not in a canonical supercell should be stored in a list or some other data structure.

Iterating over a [ConfigurationSet](https://prisms-center.github.io/CASMcode_pydocs/libcasm/configuration/2.0/reference/libcasm/_autosummary/libcasm.configuration.ConfigurationSet.html#configurationset) yields [ConfigurationRecord](https://prisms-center.github.io/CASMcode_pydocs/libcasm/configuration/2.0/reference/libcasm/_autosummary/libcasm.configuration.ConfigurationRecord.html#configurationrecord).

In [ ]:
# Iterate over the first three ConfigurationRecord
# in the ConfigurationSet and print the record
for i, record in enumerate(project.enum.last.configuration_set):
    print(record)
    if i == 2:
        break


#### Conversion to structure

The [Configuration.to_structure]() methods convert a CASM [Configuration]() to a CASM [Structure](). A Structure:

- represents a crystal structure with a 3d lattice,
- is not restricted to the DoF values allowed by a prim,
- has built in methods for conversions to and from VASP POSCAR format.

In [ ]:
# Iterate over the first three ConfigurationRecord
# in the ConfigurationSet and print the record
for i, record in enumerate(project.enum.last.configuration_set):
    name = record.configuration_name
    structure = record.configuration.to_structure()
    poscar_str = structure.to_poscar_str(title=name)
    
    print("~~~")
    print(f"Configuration: {name}")
    print(f"Structure: {structure}")
    print(f"POSCAR:\n{poscar_str}", end="")
    if i == 2:
        break

#### Filtered enumeration

A custom filter function may be used to filter configurations during enumeration. Here we:

- use [ConfigCompositionCalculator]() to calculate the number of each type of atom in the supercell,
- keep configurations that have exactly 2 Ge,
- use ``dry_run=True``so the enumeration is not committed automatically.

In [ ]:
# Enumerate configurations:
# - in supercells with volume 1 to 3
# - with exactly 2 Ge atoms in the supercell

from libcasm.configuration import (
    Configuration,
    SupercellRecord,
)
from casm.project import EnumData

# Get the casm.project.ConfigCompositionCalculator
comp = project.chemical_composition

# Get the index of Ge in the composition arrays
i_Ge = comp.components.index("Ge")

# Print each check?
verbose_checks = True


def filter_f(config: Configuration, enum_data: EnumData) -> bool:
    """Return True to include; False to exclude"""
    
    # Get number of Ge in the supercell
    N_Ge = comp.per_supercell(config)[i_Ge]
    
    # Print info about the config being checked
    if verbose_checks:
        record = SupercellRecord(config.supercell)
        print(
            f"~check~ {record.supercell_name}",
            config.occupation,
            f"include?: {N_Ge == 2}",
        )
    return N_Ge == 2

project.enum.occ_by_supercell(
    max=3, 
    min=1, 
    filter_f=filter_f,
    verbose=True,
    dry_run=True,
)

### Acting on enumeration data

This section provides a reference for various actions that can be performed on enumerations, and may be skipped for the Si-Ge demonstration project.

#### Get an enumeration by id

- Also, update enumeration metadata and commit.

In [ ]:
enum_data = project.enum.get("supercells_by_volume.1")
enum_data.meta = {"desc": "Initial supercell enumeration"}
enum_data.commit()
print(enum_data)

#### List all enumerations

- Print a summary of each enumeration in the project

In [ ]:
project.enum.list()

#### Copy an enumeration

- Will raise if the destination enumeration already exists

In [ ]:
project.enum.copy(
    src_id="supercells_by_volume.2", 
    dest_id="supercells_by_volume.3",
)
project.enum.list()

#### Merge enumerations

- Supercells and configurations in source enumeration sets are inserted into the destination enumeration sets.
- Supercells and configurations in source enumeration lists are appended to the destination enumeration lists if they are not already present.

In [ ]:
project.enum.merge(
    src_id="supercells_by_volume.1",
    dest_id="supercells_by_volume.3",
)
project.enum.list()

#### Remove an enumeration

- Will raise if the enumeration does not exist

In [ ]:
project.enum.remove("supercells_by_volume.3")
project.enum.list()

## Calculation

The calculation process described here involves the following conversions:

    libcasm.configuration.Configuration 
    -> libcasm.xtal.Structure 
    -> ase.Atoms 
    -> VASP input files (POSCAR, KPOINTS, INCAR, POTCAR)

Template VASP files will be stored in:

    <project>/calculation_settings/calctype.<calctype_id>/

The calculation directories created will be:

    <enum_dir>/training_data/<configname>/calctype.<calctype_id>/

Where
    <enum_dir> = <project>/enumerations/enum.<enum_id>/

In [ ]:
# TODO:
# # Calculation type ID
# calctype_id = "vasp.relax.default"

# # Location for calculation setings & VASP template files:
# calc_settings_dir = project.dir.calc_settings_dir(id=calctype_id)

### Construct VASP input files

## Composition axes

### Introduction

In a crystal with a fixed number of sites, the number of species occupying the same sublattice are not indepedent. In general, a crystal occupied by $s$ component species will have $k<s$ independent compositions. CASM converts between compositions expressed as number of species per unit cell and compositions in terms of independent "parametric composition axes" using:

\begin{align}
    \vec{n} &= \vec{n}_0 + \mathbf{Q} \vec{x} \\
    \vec{x} &= \mathbf{R}^{\mathsf{T}} (\vec{n} - \vec{n}_0)
\end{align}


where:

- $\vec{n}$: Vector of shape=($s$,), the number of each component species
  per unit cell (*mol_composition*).
- $\vec{x}$: Vector of shape=($k$,), The composition along each composition axis when referenced to the origin composition (*param_composition*).
- $\vec{n}_0$: Vector of shape=($s$,), The origin in composition space, as the number of each component species per unit cell.
- $Q$: Matrix of shape=($s$, $k$), with columns representing
the change in composition per unit cell going one unit distance along each independent composition axis.
- $R$: Matrix of shape=($s$, $k$), such that $\mathbf{R}^{\mathsf{T}}\mathbf{Q} = \mathbf{Q}^{\mathsf{T}}\mathbf{R} = \mathbf{I}$.

The "parametric composition axes" are the columns of $Q$, $\vec{q}_i$. Due to preservation of the number of sites per unit cell, $\sum_{i} Q_{ij} = 0$. If vacancies are allowed, they are included as a component species.

### Print standard parametric composition axes

When a CASM project is initialized, a set of standard choices for the parametric axes are determined and stored in the [Project.chemical_composition_axes] attribute. Printing the chemical_composition_axes results in a table summarizing the possible choices:

In [ ]:
# print possible axes 
print(project.chemical_composition_axes)

### Select default parametric composition axes

To select a particular choice to be used as the default for calculating parametric compositions, use [set_current_axes]():


In [ ]:
# select axes with <key>, unset with None
project.chemical_composition_axes.set_current_axes(1)

# print possible axes and formulas for current choice 
print(project.chemical_composition_axes)

# commit current choice 
project.chemical_composition_axes.commit()

Notes:

- In CASM v2, by default the parametric composition axes are "normalized" in the sense that a unit distance along that axis corresponds to a change in occupation of one site per unit cell. In CASM v1, the standard composition axes were not normalized.
- The term "endmember" usually refers to the extreme compositions in a solid solution. In the context of CompositionConverter, the term "end member composition" is used to mean the composition one unit distance along a parametric composition axis, $\vec{n}_0 + \vec{q}_i$.
- When printing formulas, the characters "a", "b", "c", etc. are used to represent the parametric compositions, $x_1$, $x_2$, $x_3$, etc.
- When referring to parametric composition axes, the characters "a", "b", "c", etc. are used to represent the parametric composition axes, $\vec{q}_1$, $\vec{q}_2$, $\vec{q}_3$, etc.